# SOFT7 Data Source

This notebook contains examples related to the SOFT7 data source.
How to create, manage, resolve, and use SOFT7 data sources.

## Generate a SOFT7 Data Source

SOFT7 Data Source instances can be generated based on information from the following parts:

1. Data source (DB, File, Webpage, ...).
2. Generic data source parser.
3. Data source parser configuration.
4. SOFT7 entity (data model).

Parts 2 and 3 are together considered to produce the "specific parser".
Parts 1 through 3 are provided as a collective, based on the [`ResourceConfig`](https://emmc-asbl.github.io/oteapi-core/latest/all_models/#oteapi.models.ResourceConfig) from [OTEAPI Core](https://emmc-asbl.github.io/oteapi-core/).

### Resource configuration

The resource configuration, originally based on the [Data Catalog Vocabulary (DCAT)](https://www.w3.org/TR/vocab-dcat-3/), is in this case a small set of data catalog fields mapped to resource-specific values:

- `downloadUrl` or `accessUrl`:
  - `downloadUrl`: The URL of the downloadable file in a given format. E.g. CSV file or RDF file.

    Usage: `downloadURL` _SHOULD_ be used for the URL at which this distribution is available directly, typically through a HTTPS GET request or SFTP.
  - `accessUrl`: A URL of the resource that gives access to a distribution of the dataset. E.g. landing page, feed, SPARQL endpoint.

    Usage: `accessURL` _SHOULD_ be used for the URL of a service or location that can provide access to this distribution, typically through a Web form, query or API call.  
    `downloadURL` is preferred for direct links to downloadable resources.
- `mediaType`: The media type of the distribution as defined by IANA [[IANA-MEDIA-TYPES](https://www.w3.org/TR/vocab-dcat-2/#bib-iana-media-types)].

  Usage: This property _SHOULD_ be used when the media type of the distribution is defined in IANA [[IANA-MEDIA-TYPES](https://www.w3.org/TR/vocab-dcat-2/#bib-iana-media-types)].
- `accessService`: A data service that gives access to the distribution of the dataset.

It is worth noting that the resource configuration **MUST** contain either `downloadUrl` _and_ `mediaType` **or** `accessUrl` _and_ `accessService`.
It may contain any combination otherwise, but a minimum of one of the two combinations is required.

The part described up to now defines the "data source" part of the SOFT7 data source.
Furthermore, based on either `mediaType` or `accessService` the "generic parser" part of the SOFT7 data source is determined.

To supply a parser configuration, one needs to know the generic parser that will be used, as well as the specific data to be retrieved.
Under "normal" circumstances, the parser configuration is stored alongside a reference to the data source for easy reusability.

Finally, a SOFT7 entity (or data model) is required to base the generated SOFT7 Data Source instance on.
Again, under "normal" circumstances, the SOFT7 entity is stored alongside a reference to the data source for easy reusability.

### Example of generating a SOFT7 Data Source

The following example shows how to generate a SOFT7 Data Source instance based on the parts described above.

In [1]:
import logging

logging.getLogger("s7").addHandler(logging.StreamHandler())

In [1]:
from s7.factories import create_entity

OPTIMADEStructure = create_entity("http://onto-ns.com/meta/1.0/OPTIMADEStructure")
print(OPTIMADEStructure.model_fields["properties"].annotation.model_fields)

{'type': FieldInfo(annotation=str, required=True, title='type', description="The name of the type of an entry. Must always be 'structures'.", json_schema_extra={}), 'attributes': FieldInfo(annotation=SOFT7DataSource, required=True, title='attributes', description='The attributes used to represent a structure, e.g. unit cell, atoms, positions.', json_schema_extra={}), 'id': FieldInfo(annotation=str, required=True, title='id', description="An entry's ID as defined in section Definition of Terms.", json_schema_extra={})}


In [2]:
import os

from s7.factories import create_datasource

os.environ["OTELIB_DEBUG"] = "true"

datasource = create_datasource(
    entity="http://onto-ns.com/meta/1.0/OPTIMADEStructure",
    resource_config={
        "downloadUrl": "https://optimade.materialsproject.org/v1/structures/mp-1228448",
        "mediaType": "application/json",
    },
)


Client <otelib.client.OTEClient object at 0x7fd90354ed00>
Data resource <otelib.backends.services.dataresource.DataResource object at 0x7fd90341bfd0>
Result {'optimade_config': None, 'optimade_response_model': ['optimade.models.optimade_json', 'Success'], 'optimade_response': {'data': {'id': 'mp-1228448', 'type': 'structures', 'attributes': {'immutable_id': '645d307dbcd30f748b48eefb', 'last_modified': '2021-02-10T01:38:17Z', 'elements': ['Al', 'O'], 'nelements': 2, 'elements_ratios': [0.4, 0.6], 'chemical_formula_descriptive': 'Al2O3', 'chemical_formula_reduced': 'Al2O3', 'chemical_formula_hill': 'Al2O3', 'chemical_formula_anonymous': 'A3B2', 'dimension_types': [1, 1, 1], 'nperiodic_dimensions': 3, 'lattice_vectors': [[-1.508747, -2.6132, 0.000129], [-1.508747, 2.6132, -0.000129], [0.0, 0.000184, -7.574636]], 'cartesian_site_positions': [[-1.508747, 2.6132805930160004, -5.469387346584], [-1.508747, -2.613096593016, -2.105248653416], [-1.508747, -0.8710047307360002, -6.063801547042], [-

In [19]:
datasource

OPTIMADEStructure(type=<function _get_data.<locals>.__get_data at 0x7f329c5915e0>, attributes=<function _get_data.<locals>.__get_data at 0x7f32a06289d0>, id=<function _get_data.<locals>.__get_data at 0x7f3284fc3700>)